# **Store Sales Forecasting with RNNs** 📈📉
# 1st part - Data Analysis 

## Introduction ✏️

Time series forecasting is one of the most important tasks in the world of business. It is a very complex task, and it is not always possible to predict the future. But we can build ML models to do so. One of the best ways to do it is to use recurrent neural networks (RNNs), which can handle time series data pretty well because they keep a memory state of the previous time steps.

To apply this concept, we will use the [Store Sales - Time Series Forecasting](https://www.kaggle.com/c/store-sales-time-series-forecasting/data) to predict the sales of a store in the next two weeks. We will read, manipulate and visualize the data, and then build a model to predict the sales. 

In this first notebook, we will analyze the data and feature engineer it. On the next notebook, we will build the model and apply it. You can check the next notebook in url. Let's get started!

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-✏️" data-toc-modified-id="Introduction-✏️-1">Introduction ✏️</a></span></li><li><span><a href="#Dependencies-👪" data-toc-modified-id="Dependencies-👪-2">Dependencies 👪</a></span></li><li><span><a href="#Reading-Data-📖" data-toc-modified-id="Reading-Data-📖-3">Reading Data 📖</a></span><ul class="toc-item"><li><span><a href="#Train-Data" data-toc-modified-id="Train-Data-3.1">Train Data</a></span></li><li><span><a href="#Test-Data" data-toc-modified-id="Test-Data-3.2">Test Data</a></span></li><li><span><a href="#Sample-Submission" data-toc-modified-id="Sample-Submission-3.3">Sample Submission</a></span></li><li><span><a href="#Stores-Data" data-toc-modified-id="Stores-Data-3.4">Stores Data</a></span></li><li><span><a href="#Oil-Data" data-toc-modified-id="Oil-Data-3.5">Oil Data</a></span></li><li><span><a href="#Transactions-Data" data-toc-modified-id="Transactions-Data-3.6">Transactions Data</a></span></li><li><span><a href="#Holiday-Events-Data" data-toc-modified-id="Holiday-Events-Data-3.7">Holiday Events Data</a></span></li><li><span><a href="#Additional-Notes" data-toc-modified-id="Additional-Notes-3.8">Additional Notes</a></span></li></ul></li><li><span><a href="#Understanding-the-Data-📚" data-toc-modified-id="Understanding-the-Data-📚-4">Understanding the Data 📚</a></span><ul class="toc-item"><li><span><a href="#Merging-our-tables" data-toc-modified-id="Merging-our-tables-4.1">Merging our tables</a></span><ul class="toc-item"><li><span><a href="#Store-metadata" data-toc-modified-id="Store-metadata-4.1.1">Store metadata</a></span></li><li><span><a href="#Oil-data" data-toc-modified-id="Oil-data-4.1.2">Oil data</a></span></li><li><span><a href="#Transactions" data-toc-modified-id="Transactions-4.1.3">Transactions</a></span></li></ul></li><li><span><a href="#Holiday-Events" data-toc-modified-id="Holiday-Events-4.2">Holiday Events</a></span></li></ul></li><li><span><a href="#Analyzing-our-Data-📊" data-toc-modified-id="Analyzing-our-Data-📊-5">Analyzing our Data 📊</a></span><ul class="toc-item"><li><span><a href="#Does-Oil-Price-really-affects-sales?" data-toc-modified-id="Does-Oil-Price-really-affects-sales?-5.1">Does Oil Price really affects sales?</a></span></li><li><span><a href="#Where-are-located-the-stores-with-the-highest-sales?" data-toc-modified-id="Where-are-located-the-stores-with-the-highest-sales?-5.2">Where are located the stores with the highest sales?</a></span></li><li><span><a href="#Which-Categories-Sell-More?" data-toc-modified-id="Which-Categories-Sell-More?-5.3">Which Categories Sell More?</a></span></li><li><span><a href="#How-many-stores-are-in-each-city?" data-toc-modified-id="How-many-stores-are-in-each-city?-5.4">How many stores are in each city?</a></span></li><li><span><a href="#Are-holidays-important?" data-toc-modified-id="Are-holidays-important?-5.5">Are holidays important?</a></span></li></ul></li></ul></div>

## Dependencies 👪

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os

In [ ]:
# custom template for plotly
custom_template_go = {
    "layout": go.Layout(
        font={
            "size": 12,
            "color": "#666",
        },
        title={
            "font": {
                "family": "Times New Roman",
                "size": 18,
                "color": "#666",
            },
        },
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff",
        xaxis={
            "showgrid": False,
            "zeroline": False,
        },
        yaxis={
            "showgrid": False,
            "zeroline": False,
        },
        margin=dict(b=20,r=60,l=70,t=115),
    )
}

## Reading Data 📖

In [ ]:
print("Path of our data files:")
for path in os.listdir("../input/store-sales-time-series-forecasting"):
    print(os.path.join("../input/store-sales-time-series-forecasting", path))

### Train Data

- The training data, comprising time series of features `store_nbr`, `family`, and `onpromotion` as well as the target `sales`.
- `store_nbr` identifies the store at which the products are sold.
- `family` identifies the type of product sold.
- `sales` gives the total sales for a product `family` at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- `onpromotion` gives the total number of items in a product family that were being promoted at a store at a given date.

In [ ]:
train_data = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv", index_col="id")
train_data_old = train_data.copy()
train_data.head()

### Test Data

- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.

In [ ]:
test_data = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv", index_col="id")
test_data.head()

### Sample Submission

- A sample submission file in the correct format.

In [ ]:
sample_submission = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")
sample_submission.head()

### Stores Data

- Store metadata, including `city`, `state`, `type`, and `cluster`.
- `cluster` is a grouping of similar stores.

In [ ]:
stores_data = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
stores_data.rename(columns={"type":"type_of_store"}, inplace=True)
stores_data.head()

### Oil Data

- Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [ ]:
oil_data = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
oil_data.head()

### Transactions Data

- Number of complete daily web transactions for each store.

In [ ]:
transactions_data = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")
transactions_data.head()

### Holiday Events Data

- Holidays and Events, with metadata
- **NOTE**: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [ ]:
holidays_data = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")
holidays_data.rename(columns={"type":"type_of_event"}, inplace=True)
holidays_data.head()

### Additional Notes

- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

All these data descriptions are not mine, they were provided by the Kaggle team. I put them here so we can have a better understanding of the data. You can check it out in the [Kaggle competition page](https://www.kaggle.com/c/store-sales-time-series-forecasting/data).

## Understanding the Data 📚
In this section, we will look at the data and see what we can learn from it. We will merge different tables, create new features, and explore the data with different visualizations.

Firstly, let's merge our tables.

### Merging our tables

We will merge our training/test data with our store metadata, using the `store_nbr` column as the key.

#### Store metadata

In [ ]:
train_data = pd.merge(train_data, stores_data, on="store_nbr", how="left")
test_data = pd.merge(test_data, stores_data, on="store_nbr", how="left")
train_data.head()

Now, let's do the same with our oil data. But firstly, let's see if our data has too many NaN values.

#### Oil data

In [ ]:
print("Shape of oil_data:" ,oil_data.shape)

In [ ]:
print("Number of null values in oil_data:")
oil_data.isnull().sum()

We have only 43 NaN values in the oil data. But if we merge our table with the oil data, we will have way more NaN values. Let's analyze the best way to impute the NaN values and then merge our tables.

In [ ]:
fig = go.Figure(go.Scatter(x=oil_data.date, y=oil_data.dcoilwtico, mode="lines", marker_color="brown"))
fig.update_layout(yaxis_title="Price",
                  yaxis_showgrid=False,
                  template=custom_template_go,
                  title={'text':"<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>from 2013 to 2016<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                 )
fig.show()

We can see that besides the oil price is unstable, the oil price is decreasing, and besides some NaN values, we can see a continuous curve along time. So we can impute the NaN values with the last known value, to maintain our continuity.

In [ ]:
oil_data.fillna(method="bfill", inplace=True)

In [ ]:
fig = go.Figure(go.Scatter(x=oil_data.date, y=oil_data.dcoilwtico, mode="lines", marker_color="brown"))
fig.update_layout(yaxis_title="Price",
                  template=custom_template_go, 
                  title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>with replaced <span style='color:#6D83AA'>NaN </span>values<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                 )
fig.show()

In [ ]:
train_data = pd.merge(train_data, oil_data, on="date", how="left")
test_data = pd.merge(test_data, oil_data, on="date", how="left")

train_data.head()

#### Transactions

In [ ]:
print("nº of null values in transactions_data:")
transactions_data.isnull().sum()

In [ ]:
train_data = pd.merge(train_data, transactions_data, on=["date", "store_nbr"], how="left")
test_data = pd.merge(test_data, transactions_data, on=["date", "store_nbr"], how="left")

train_data.head()

### Holiday Events

In [ ]:
print("nº of null values in holidays_data:")
holidays_data.isnull().sum()

In [ ]:
holidays_data = holidays_data.groupby("date")[["type_of_event", "transferred", "locale", "locale_name"]].agg(lambda x: x.unique() if x.nunique() > 1 else list(x)) # adding multiple holidays at the same day an unique row

holidays_data["n_holidays"] = holidays_data.type_of_event.map(lambda x: len(x))

In [ ]:
train_data = pd.merge(train_data, holidays_data, on="date", how="left", )
test_data = pd.merge(test_data, holidays_data, on="date", how="left")

Now that we have just one table, let's analyze what columns are useful and gain insights about our data.

## Analyzing our Data 📊

### Does Oil Price really affects sales?
Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices. But does this reflects in the sales of our products? 

In [ ]:
# converting date to datetime
train_data.date = pd.to_datetime(train_data.date)
test_data.date = pd.to_datetime(test_data.date)

In [ ]:
# grouping by month sales and oil price
sales_oil_by_month = train_data[["date", "sales", "dcoilwtico"]].copy()
sales_oil_by_month = sales_oil_by_month.groupby(pd.Grouper(key="date", freq="M")).mean()
sales_oil_by_month.fillna(method="ffill", inplace=True) # filling missing values with previous value

In [ ]:
fig = make_subplots(rows=2, cols=1)
# adding traces
fig.add_trace(go.Scatter(x=sales_oil_by_month.index, y=sales_oil_by_month.sales, mode="lines", name="Sales", marker_color="limegreen"), row=1, col=1)
fig.add_trace(go.Scatter(x=sales_oil_by_month.index, y=sales_oil_by_month.dcoilwtico, 
                         mode="lines", name="Oil Price", marker_color="brown"), row=2, col=1)
# updating axis titles
fig.update_xaxes(showgrid=False, row=1, col=1)
fig.update_xaxes(title_text="Date", showgrid=False, row=2, col=1)
fig.update_yaxes(title_text="Sales", showgrid=False, row=1, col=1)
fig.update_yaxes(title_text="Oil Price", showgrid=False, row=2, col=1)
# add vertical line in the plot
fig.update_layout(
        shapes=[
        dict(type="line", x0=pd.to_datetime("2014-12-31"), xref="x", y0=0, yref="y", x1=pd.to_datetime("2014-12-31"),
        y1=sales_oil_by_month.sales.max(),
        line=dict(color="black", width=2, dash="dot")),
        dict(type="line", x0=pd.to_datetime("2014-12-31"), xref="x2", y0=0, yref="y2", x1=pd.to_datetime("2014-12-31"),
        y1=sales_oil_by_month.dcoilwtico.max(),
        line=dict(color="black", width=2, dash="dot")),
        ],
        annotations=[
                dict(x=pd.to_datetime("2015-10-31"), y=sales_oil_by_month.sales.max()+100, xref="x", yref="y",
        text="When <span style='color:brown'>Oil Price</span> starts to decrease, <span style='color:limegreen'>sales</span> start to increase", showarrow=False, font=dict(color="black", size=12))
        ]
)
# updating figure layout
fig.update_layout(template=custom_template_go,
                  title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Sales and Oil Price</span><br>from 2013 to 2016<br><br>",
                  'y':0.85, 'x':0.5,
                  'xanchor': 'center', 'yanchor': 'top'},
                    )
fig.show()

### Where are located the stores with the highest sales?

In [ ]:
by_state = train_data.groupby(["state"]).sum().sort_values(by="sales", ascending=True)
by_state.index = by_state.index.astype(str)

fig = go.Figure(go.Bar(y=by_state.index, x=by_state.sales, 
                       orientation="h", marker_color="limegreen",
                       text=by_state.sales, textposition="outside", texttemplate='%{text:.2s}'))
fig.update_layout(template=custom_template_go,
                    margin=dict(b=20,r=60,l=80,t=115),
                    title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Sales by State ($)</span><br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
                    )
fig.show()

We can clearly see that the states locations are the very important factors in the sales. But what about the cities?

In [ ]:
by_city = train_data[["city", "state", "sales", "store_nbr"]].groupby(["city"]).agg({'sales':'sum', 'state':'first', 'store_nbr':'nunique'}).sort_values(by="sales", ascending=False) # grouping sales by city and state

fig = px.bar(y=by_city.index, x=by_city.sales, orientation="h", color=by_city.state, 
             text=by_city.sales, 
             color_discrete_sequence=["#000000", "#145214", "#d6f5d6", "#2eb82e", "#85e085", "#00ff00", "#669900", "#b3ff1a", "#c6ff1a", "#00ff99", "#00b3b3", "#006666", "#003333", "#669999", "#ffbf00", "#663300"],
             labels={"color": "State"})

# updating text position and formatting
fig.update_traces(texttemplate='%{text:.2s}', textposition="outside")
# add text annotation
fig.add_annotation(x=by_city.sales.max()/2.5, y=by_city.index[-1], text="The state with most cities has only 4,<br>and not all cities <br>in their respective state has the same sales pattern", showarrow=False, font=dict(color="black", size=12), xanchor="left", yanchor="top")
# updating figure layout
fig.update_layout(template=custom_template_go,
                    plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                    margin=dict(b=20,r=60,l=70,t=115),
                    title={'text': '<span style="font-size:45px; color=#444; font-family:Times New Roman">Sales by City ($)</span><br>grouped by each state<br><br>',
                            'y':0.85, 'x':0.5,
                            'xanchor': 'center', 'yanchor': 'top'}, 
                    )
fig.show()

In [ ]:
train_data.drop(["state"], axis=1, inplace=True) # dropping state column
test_data.drop(["state"], axis=1, inplace=True) 

### Which Categories Sell More?

In [ ]:
by_family = train_data[["family", "sales"]].groupby("family").sum().sort_values(by="sales")
by_family.index = by_family.index.str.title() # capitalizing family names

fig = go.Figure(go.Bar(y=by_family.index[-10:], x=by_family.sales[-10:], 
                       orientation="h", marker_color="limegreen",
                       text=by_family.sales[-10:], textposition="outside", texttemplate='%{text:.2s}'))
fig.update_layout(template=custom_template_go,
                    margin=dict(b=20,r=40,l=200,t=115),
                    title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Top 10 Product Categories ($)</span><br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
                    xaxis_title="Sales",
                    yaxis_title="Product Categories",
                    )
fig.show()

In [ ]:
other = by_family[:-10].sum()
top10 = by_family[-10:].sum()

fig = go.Figure()
fig.add_trace(go.Pie(labels=["Top 10 Categories", "Others"], values=[top10.sales, other.sales], marker_colors=["2eb82e", "#d6f5d6"],))
fig.update_layout(template=custom_template_go,
                  title={'text': "<span style='font-size:45px; font-family:Times New Roman'>Top 10 Categories vs. Rest</span><br><span style='color:#2eb82e'>Top 10</span> represents <span style='color:#2eb82e'>93%</span> of the sales<br><br>",
                    'y':0.85, 'x':0.5,
                    'xanchor': 'center', 'yanchor': 'top'},
)
fig.show()

We will only mantain the top 10 categories, and label the others as "Others".

In [ ]:
other = by_family[:-10]
other.index = other.index.str.upper()
train_data["family"] = train_data.family.map(lambda x: "Others" if x in other.index else x)
test_data["family"] = test_data.family.map(lambda x: "Others" if x in other.index else x)

### How many stores are in each city?

In [ ]:
by_city[["sales", "store_nbr"]].head(10)

We can see that quantity of stores in each city is an important factor in the sales only in the two cities that have the highest sales.

### Are holidays important?

In [ ]:
train_data.n_holidays.fillna(0, inplace=True)
test_data.n_holidays.fillna(0, inplace=True)

In [ ]:
print("nº of holidays per day:")
train_data.n_holidays.value_counts()

In [ ]:
print("Average sales per nº of holidays:")
train_data.groupby("n_holidays").sales.mean()

It appears that two holidays are important in the sales. Probably because more parts of the country has at least one holiday.

In [ ]:
train_data.drop(["type_of_event", "transferred", "locale", "locale_name"], axis=1, inplace=True)
test_data.drop(["type_of_event", "transferred", "locale", "locale_name"], axis=1, inplace=True)

### Saving our cleaned data

In [ ]:
train_data.to_csv("./train_data_cleaned.csv", index=False)
test_data.to_csv("./test_data_cleaned.csv", index=False)